## Встановимо бібліотеку surprise

In [ ]:
pip install scikit-surprise==1.1.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162733 sha256=d24fd2204de2e157f851aa976a4df755f6293155739eaff905ddd8f070083848
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


## Імпортуємо потрібні бібліотеки

In [ ]:
import pandas as pd
from surprise import SVD, SVDpp, NMF, accuracy, Dataset, Reader
from surprise.model_selection import cross_validate, train_test_split

## Завантажимо необхідні датасети

Посилання на використаний датасет: https://files.grouplens.org/datasets/movielens/ml-latest-small.zip

In [ ]:
movies_url = 'https://drive.google.com/file/d/1zgRfJatNf4Ivn1EiaL3FkbI4ZL92lFxT/view?usp=drive_link'
ratings_url = 'https://drive.google.com/file/d/1_6f8HyCQFdnLpxEmAPGd2lMJi1sQ1Leg/view?usp=drive_link'
# tags_url = 'https://drive.google.com/file/d/1yHPCEq-uTC2LCDgOOd2nFMr1W81VeywS/view?usp=drive_link'
# links_url = 'https://drive.google.com/file/d/1LHj6rw1O0OdUi912eTeqM1eNGwejlpLM/view?usp=drive_link'

In [ ]:
def read_data(url: str):
    file_id = url.split('/')[-2]
    url = f'https://drive.usercontent.google.com/download?id={file_id}&export=download&authuser=0'
    return pd.read_csv(url)

In [ ]:
movies_data = read_data(movies_url)
movies_data.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [ ]:
ratings_data = read_data(ratings_url)
ratings_data.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
ratings_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


## Підготуємо дані

In [ ]:
rating_train = ratings_data.drop(['timestamp'], axis=1)
rating_train.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [ ]:
reader = Reader()
data = Dataset.load_from_df(rating_train, reader)

## Оцінимо три алгоритми (SVD, SVD++ та NMF) за допомогою крос-валідації

In [ ]:
algo_svd = SVD()
cross_validate(algo_svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8735  0.8806  0.8784  0.8606  0.8780  0.8742  0.0072  
MAE (testset)     0.6697  0.6739  0.6757  0.6612  0.6756  0.6712  0.0055  
Fit time          1.47    1.51    1.51    1.88    1.59    1.59    0.15    
Test time         0.46    0.12    0.31    0.21    0.12    0.24    0.13    


{'test_rmse': array([0.87348132, 0.88063224, 0.8784195 , 0.86061789, 0.87797584]),
 'test_mae': array([0.6696513 , 0.6738668 , 0.67565803, 0.66117145, 0.67556579]),
 'fit_time': (1.4704861640930176,
  1.5100891590118408,
  1.5098419189453125,
  1.8753149509429932,
  1.5903384685516357),
 'test_time': (0.4615323543548584,
  0.12039399147033691,
  0.30766749382019043,
  0.20872855186462402,
  0.11583304405212402)}

In [ ]:
algo_svdpp = SVDpp()
cross_validate(algo_svdpp, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8699  0.8671  0.8682  0.8544  0.8534  0.8626  0.0072  
MAE (testset)     0.6651  0.6665  0.6635  0.6551  0.6549  0.6610  0.0050  
Fit time          90.02   89.23   103.03  107.34  103.19  98.56   7.46    
Test time         13.28   12.40   13.68   13.19   12.53   13.02   0.48    


{'test_rmse': array([0.8699124 , 0.86713011, 0.86816933, 0.85439326, 0.85344621]),
 'test_mae': array([0.66507627, 0.66649106, 0.66352372, 0.65505409, 0.65488464]),
 'fit_time': (90.02253985404968,
  89.22515797615051,
  103.03259992599487,
  107.34046912193298,
  103.19279527664185),
 'test_time': (13.27786374092102,
  12.400564670562744,
  13.67893362045288,
  13.194498777389526,
  12.534932136535645)}

In [ ]:
algo_nfm = NMF()
cross_validate(algo_nfm, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9297  0.9192  0.9136  0.9119  0.9195  0.9188  0.0063  
MAE (testset)     0.7106  0.7038  0.7002  0.7002  0.7043  0.7038  0.0038  
Fit time          2.75    2.73    3.46    2.86    2.92    2.94    0.27    
Test time         0.10    0.10    0.10    0.10    0.33    0.15    0.09    


{'test_rmse': array([0.92973952, 0.91923043, 0.91357478, 0.91187169, 0.91951441]),
 'test_mae': array([0.71061911, 0.70382192, 0.70016948, 0.70018224, 0.70431745]),
 'fit_time': (2.749907970428467,
  2.726344585418701,
  3.458089828491211,
  2.864342212677002,
  2.9232592582702637),
 'test_time': (0.09924006462097168,
  0.09735393524169922,
  0.10157537460327148,
  0.10363912582397461,
  0.33356785774230957)}

## Здійснимо прогнози на тестовому наборі за допомогою алгоритмів (SVD, SVD++ та NMF)

In [ ]:
trainset, testset = train_test_split(data, test_size=0.2)

In [ ]:
algo_svd.fit(trainset)
pred = algo_svd.test(testset)
print(accuracy.rmse(pred))
print(accuracy.mae(pred))

RMSE: 0.8776
0.8775611333945177
MAE:  0.6757
0.6757147717825678


In [ ]:
algo_svdpp.fit(trainset)
pred = algo_svdpp.test(testset)
print(accuracy.rmse(pred))
print(accuracy.mae(pred))

RMSE: 0.8658
0.865785746408609
MAE:  0.6636
0.6636109677930243


In [ ]:
algo_nfm.fit(trainset)
pred = algo_nfm.test(testset)
print(accuracy.rmse(pred))
print(accuracy.mae(pred))

RMSE: 0.9198
0.9198365336198092
MAE:  0.7077
0.7077313346708708


## Висновок

Порівнявши три алгоритми SVD, SVD++ та NMF можна зрозуміти, що:
- за показникоми Root Mean Squared Error (RMSE) та Mean absolute error (MAE) оптимальнішим є SVD++, але треба зауважити, що і виконується він набагато довше, на датасеті 100 тис записів, а отже для більш великих наборів даних, треба мати більш потужніше залізо.
- на другому місці SVD;
- відповідно, на третьому NMF.